In [64]:
from data import data_generation, batch_creator, gp_kernels
import sklearn.gaussian_process as gp 
import numpy as np
import scipy

%matplotlib inline

In [3]:
pad_pos_tr, pad_pos_te, pad_y_fren_tr, pad_y_fren_te, _, df_te = data_generation.data_generator_for_gp_mimick_gpt(50000, gp_kernels.rbf_kernel)

In [38]:
indices_tr = np.random.choice(np.arange(0, len(pad_pos_tr.reshape(-1)), 1), 10000)
x_tr = pad_pos_tr.reshape(-1)[indices_tr]; y_tr = pad_y_fren_tr.reshape(-1)[indices_tr]
x_te = pad_pos_te.reshape(-1)[:(59 * 100)]; y_te = pad_y_fren_te.reshape(-1)[:(59 * 100)]

In [ ]:
σ = 1; ν = 1; λ = np.exp(-1)

In [63]:
kernel = ν**2 * gp.kernels.RBF(length_scale=λ) + gp.kernels.WhiteKernel(noise_level=σ)
model = gp.GaussianProcessRegressor(kernel=kernel)
model.fit(x_tr.reshape(-1, 1), y_tr.reshape(-1, 1))

In [70]:
# Online GP sequence learning  
online = False; mse = 0
# μ = []; σ = []
m = int(x_te.shape[0] / 59)
for j in range(0, m):
    λ=np.exp(-1); ν=1; σ=1
    kernel = ν**2 * gp.kernels.RBF(length_scale=λ) + gp.kernels.WhiteKernel(noise_level=σ)
    model = gp.GaussianProcessRegressor(kernel=kernel)
    xx_tr =  x_tr.reshape(-1, 1)[:(40 + 59 * j)]; yy_tr = y_tr.reshape(-1, 1)[:(40 + 59 * j)]
    model.fit(xx_tr, yy_tr)
    if online:
        pass
    else: 
        xx_te = x_tr.reshape(-1, 1)[(40 + 59 * j):(59 + 59 * j)]
        yy_te = y_te.reshape(-1, 1)[(40 + 59 * j):(59 + 59 * j)]
        μ_te, σ_te = model.predict(xx_te , return_std=True)
    
    mse += (sum((yy_te - μ_te)**2) / 19)
mse *= (1/m) 

In [71]:
mse

array([1.0973226])

In [56]:
μ_te, σ_te = model.predict(x_te.reshape(-1, 1), return_std=True)

In [59]:
sum((μ_te -  y_te.reshape(-1, 1)) ** 2) / len(μ_te)

array([1.01432648])

In [62]:
sum((y_te.mean() -  y_te.reshape(-1, 1)) ** 2) / len(μ_te)

array([1.01301103])

In [ ]:
with matplotlib.rc_context({'figure.figsize': [10,2.5]}):
    plt.scatter(x.squeeze(), y)
    plt.scatter(x, μ_tr, c='black')